## Main Scraper

In [49]:
# %%
import os
# import json # Unused
# import sys # Unused
# from sys import exit # Unused
# import numpy as np # Unused
import time
# import urllib.request # Unused
# import ssl # Unused, and the modification in the original script is generally not recommended for Selenium
from dotenv import load_dotenv
# from unicodedata import name # Unused
from selenium import webdriver
# from selenium.webdriver.common.keys import Keys # Not explicitly used, send_keys is a method of WebElement
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
# from urllib.parse import urlparse, parse_qs # Unused
import pandas as pd
# %%
load_dotenv()


True

In [50]:
# %%
### Constants
FACEBOOK_LOGIN_URL = 'https://www.facebook.com/login/'
# It's better to use environment variables or a config file for these paths
# Defaulting to your provided paths if environment variables are not set.
CHROME_DRIVER_PATH = os.getenv("CHROME_DRIVER_PATH", 'E:/Users/1167486/Local/Drivers_web/chromedriver-win64/chromedriver.exe')
CHROME_BINARY_PATH = os.getenv("CHROME_BINARY_PATH", r'E:/Users/1167486/Local/Drivers_web/chrome-win64/chrome.exe')
print(f"Using Chrome Driver Path: {CHROME_DRIVER_PATH}")
print(f"Using Chrome Binary Path: {CHROME_BINARY_PATH}")


#service = Service('E:/Users/1167486/Local/Drivers_web/chromedriver-win64/chromedriver')
#options = webdriver.ChromeOptions()
#options.binary_location = r'E:/Users/1167486/Local/Drivers_web/chrome-win64/chrome.exe'

# CSS Selectors - Highly Prone to change on Facebook
USERNAME_SELECTOR = "input[name='email']"
PASSWORD_SELECTOR = "input[name='pass']"
LOGIN_BUTTON_SELECTOR = "button[type='submit']"
# !!! IMPORTANT: This comment selector is based on CSS classes which Facebook changes frequently.
# This is the most likely part of the script to break.
# Finding stable selectors for Facebook elements is a significant challenge.
COMMENT_SELECTOR = "div[class='xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs']" # From your script, likely unstable
COMMENT_CONTAINER_SELECTOR = "div[aria-label*='Comment by'], div.x78zum5.x1n2onr6.xh8yej3" # Trying to find a div that wraps a whole comment.
USERNAME_IN_COMMENT_XPATH = ".//span[contains(@class,'x193iq5w') or contains(@class,'xt0psk2')]/a[@role='link'] | .//a[@aria-label and @role='link' and not(contains(@href,'reaction')) and not(contains(@href,'comment_id'))][1]"
# Example for timestamp (likely outdated):
TIMESTAMP_IN_COMMENT_XPATH = ".//a[contains(@href,'comment_id=') and @role='link']//span[string-length(normalize-space(text())) > 0] | .//a[contains(@href,'comment_id=') and @role='link']"
# Example for comment text (this was your original selector, ensure it's relative or specific enough)
# If the COMMENT_CONTAINER_SELECTOR is good, this might be simpler like ".//div[@dir='auto']"
COMMENT_TEXT_IN_COMMENT_XPATH = ".//div[@class='xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs'] | .//div[@dir='auto' and not(.//a[@role='link'])]"
# --- End Selectors ---
# Target page/post
# TARGET_PAGE_NAME = 'cinexcepción' # Used if you intend to search for the page first
#TARGET_POST_URL = 'https://www.facebook.com/Cinexcepcion/posts/pfbid0bhBrBUN5ZMqWDJhk3nTJm9VzNspzcX9bcLSdxJNJBsu3UVZZuq9yjVVvTmjGcDMal'
#TARGET_POST_URL = 'https://www.facebook.com/revolutapp/posts/pfbid036Ex3ZyiYbApCH3mNzt6e8KxtnnoX2Cs9GCkn4SWc8sarJPYJTvnNTRaXVJ8A7MvDl'

# Timeouts
DEFAULT_TIMEOUT = 35  # seconds for WebDriverWait

### Load environment variables
FACEBOOK_USER = os.getenv("facebook_user")
FACEBOOK_PASSWORD = os.getenv("facebook_password")
print(FACEBOOK_USER)

print(FACEBOOK_PASSWORD)

USERNAME_SELECTOR ='x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x676frb x1nxh6w3 x1sibtaa x1s688f xzsf02u'
TIMESTAMP_SELECTO ='oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw'

Using Chrome Driver Path: E:/Users/1167486/Local/Drivers_web/chromedriver-win64/chromedriver.exe
Using Chrome Binary Path: E:/Users/1167486/Local/Drivers_web/chrome-win64/chrome.exe
modelonegociosisul@gmail.com
innovacion00Modelos/


In [51]:
'''
if not FACEBOOK_USER or not FACEBOOK_PASSWORD:
    print("Error: Facebook username or password not found in .env file or environment variables.")
    # Consider exiting or raising an error: raise ValueError("Missing Facebook credentials")

## Initialize WebDriver to None for the finally block
driver = None

try:
    print("Initializing Chrome WebDriver...")
    service = Service(CHROME_DRIVER_PATH)
    options = webdriver.ChromeOptions()
    options.binary_location = CHROME_BINARY_PATH
    # Optional: Add arguments like headless mode or disabling notifications
    # options.add_argument("--headless")
    # options.add_argument("--disable-notifications")
    # options.add_argument("--lang=en-US") # Setting language can sometimes help with consistency

    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, DEFAULT_TIMEOUT)
    print("WebDriver initialized.")

    # %%
    print(f"Navigating to login page: {FACEBOOK_LOGIN_URL}")
    driver.maximize_window()
    driver.get(FACEBOOK_LOGIN_URL)

    # %%
    # The line `ssl._create_default_https_context = ssl._create_unverified_context`
    # is generally not needed for Selenium as the browser handles SSL.
    # It's also a security risk to disable SSL verification.
    # `textos = []` was unused.

    # %%
    print("Attempting to log in...")
    # Wait for the username field to be present and visible
    username_field = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, USERNAME_SELECTOR)))
    # Assuming password field is also present if username field is
    password_field = driver.find_element(By.CSS_SELECTOR, PASSWORD_SELECTOR)

    # %%
    username_field.clear()
    password_field.clear()
    print("Username and password fields cleared.")

    # %%
    username_field.send_keys(FACEBOOK_USER)
    password_field.send_keys(FACEBOOK_PASSWORD)
    print("Facebook credentials entered.")

    # %%
    login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, LOGIN_BUTTON_SELECTOR)))
    login_button.click()
    print("Login button clicked.")

    # After login, it's crucial to wait for the page to actually load.
    # This could be waiting for a specific element that only appears after successful login.
    # For example, the main feed, a navigation bar element, or your profile picture.
    # A simple time.sleep() is unreliable.
    # Example: wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "your_post_login_element_selector")))
    print("Waiting for login to process (this might take a few seconds)...")
    # Using a short sleep for now, but replace with a more robust WebDriverWait for a known element on the home/feed page.
    time.sleep(8) # Increased sleep, but ideally replace with explicit wait
    print("Login process hopefully complete.")

    # %%
    # If you need to search for a page first:
    # print(f"Searching for page: {TARGET_PAGE_NAME}")
    # search_url = f"https://www.facebook.com/search/pages/?q={TARGET_PAGE_NAME}"
    # driver.get(search_url)
    # print(f"Waiting for search results for '{TARGET_PAGE_NAME}'...")
    # wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "some_search_result_indicator_selector")))
    # time.sleep(2) # Replace with WebDriverWait

    # %%
    print(f"Navigating directly to post: {TARGET_POST_URL}")
    driver.get(TARGET_POST_URL)
    print("Waiting for post page to load...")

    # Wait for a general post container to ensure the page has started rendering.
    # You might need a more specific selector for the content you're interested in.
    try:
        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@role='article'] | //div[contains(@aria-label, 'post')] | //div[contains(@data-testid, 'post_message')]"))) # More generic post indicators
        print("Post page appears to be loading.")
    except TimeoutException:
        print("Timed out waiting for the main post content to load. The page might be structured differently or loading very slowly.")
        # Consider how to handle this: retry, log, or exit.

    # %%
    # --- Comment Extraction ---
    # WARNING: Scraping Facebook is against their Terms of Service and can lead to account restrictions.
    # Facebook's HTML structure and CSS class names change very frequently, making selectors extremely brittle.
    # The selector used here (COMMENT_SELECTOR) is highly likely to break without notice.
    # More robust methods might involve looking for ARIA roles, specific text patterns, or structural XPaths,
    # but these are also challenging to maintain for Facebook.

    print(f"Attempting to find comments using selector: {COMMENT_SELECTOR}")
    print("This may take a moment. If it times out, the selector is likely outdated or no comments are visible.")

    comments_elements = []
    try:
        # Scroll down a bit to trigger loading of comments if they are dynamically loaded
        # For more robust scrolling, you might need multiple scrolls and checks.
        print("Scrolling down slightly to help load comments...")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 3);")
        time.sleep(2) # Wait for potential dynamic loading

        # Wait for at least one comment element to be present.
        # Using presence_of_all_elements_located to get a list.
        comments_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, COMMENT_SELECTOR)))
        print(f"Found {len(comments_elements)} potential comment elements using the selector.")
    except TimeoutException:
        print(f"Timed out waiting for comments to load with selector: {COMMENT_SELECTOR}")
        print("This could be due to several reasons:")
        print("  1. No comments are present or visible on the post.")
        print("  2. The Facebook HTML structure has changed, and the CSS selector is outdated.")
        print("  3. The page/comments did not load quickly enough (network or performance issues).")
        print("  4. You might need to implement more sophisticated scrolling to load all comments.")
    except NoSuchElementException: # Should be caught by TimeoutException with presence_of_all_elements_located
        print(f"No comments found with selector: {COMMENT_SELECTOR}. The selector might be incorrect or no comments exist.")

    # %%
    comments_text_list = []
    if comments_elements:
        print("\n--- Extracted Comments ---")
        for i, comment_element in enumerate(comments_elements):
            try:
                comment_text = comment_element.text.strip()
                if comment_text:  # Process only if text is found
                    print(f"Comment {i+1}: {comment_text}")
                    comments_text_list.append(comment_text)
                else:
                    # This can happen if the div exists but contains no visible text,
                    # or if it's a container for other elements like images/reactions.
                    print(f"Comment {i+1}: (Element found, but no direct text extracted or text is empty)")
            except Exception as e_text:
                print(f"Error extracting text from comment element {i+1}: {e_text}")
        print("------------------------")
        if not comments_text_list:
            print("Although comment elements were located, no text content could be extracted from them.")
    else:
        print("No comment elements were found or successfully waited for. Cannot extract comments.")

    # Now `comments_text_list` contains the extracted comment texts.
    # You can save it to a file, process it further, etc.
    print(f"\nTotal comment texts successfully extracted: {len(comments_text_list)}")
    if comments_text_list:
        # Example: print the first few
        for i, text in enumerate(comments_text_list[:3]):
            print(f"Sample extracted comment {i}: {text[:100]}...")


except TimeoutException as e:
    print(f"A critical timeout occurred during the process: {e}")
except NoSuchElementException as e: # Should be less common with WebDriverWait unless find_element is used without wait
    print(f"A required element was not found: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    import traceback
    traceback.print_exc()
finally:
    if driver:
        print("Closing the WebDriver.")
        driver.quit()
    print("Script finished.")
'''


'\nif not FACEBOOK_USER or not FACEBOOK_PASSWORD:\n    print("Error: Facebook username or password not found in .env file or environment variables.")\n    # Consider exiting or raising an error: raise ValueError("Missing Facebook credentials")\n\n## Initialize WebDriver to None for the finally block\ndriver = None\n\ntry:\n    print("Initializing Chrome WebDriver...")\n    service = Service(CHROME_DRIVER_PATH)\n    options = webdriver.ChromeOptions()\n    options.binary_location = CHROME_BINARY_PATH\n    # Optional: Add arguments like headless mode or disabling notifications\n    # options.add_argument("--headless")\n    # options.add_argument("--disable-notifications")\n    # options.add_argument("--lang=en-US") # Setting language can sometimes help with consistency\n\n    driver = webdriver.Chrome(service=service, options=options)\n    wait = WebDriverWait(driver, DEFAULT_TIMEOUT)\n    print("WebDriver initialized.")\n\n    # %%\n    print(f"Navigating to login page: {FACEBOOK_LOGIN_

In [52]:
# %%
### Constants
FACEBOOK_LOGIN_URL = 'https://www.facebook.com/login/'
CHROME_DRIVER_PATH = os.getenv("CHROME_DRIVER_PATH", 'E:/Users/1167486/Local/Drivers_web/chromedriver-win64/chromedriver.exe')
CHROME_BINARY_PATH = os.getenv("CHROME_BINARY_PATH", r'E:/Users/1167486/Local/Drivers_web/chrome-win64/chrome.exe')

# --- CSS/XPATH Selectors ---
# !!! CRITICAL: These selectors are EXTREMELY LIKELY TO BREAK. You MUST inspect Facebook's HTML
# and update them regularly for the script to work. These are illustrative examples.
USERNAME_SELECTOR_LOGIN = "input[name='email']"
PASSWORD_SELECTOR_LOGIN = "input[name='pass']"
LOGIN_BUTTON_SELECTOR = "button[type='submit']"

# Selectors for comment extraction (relative to the comment container)
# This selector should identify the main block/div for a single comment.
# Example (likely outdated, inspect the page for current classes/attributes):
#COMMENT_CONTAINER_SELECTOR = "div[aria-label*='Comment by'], div.x78zum5.x1n2onr6.xh8yej3" # Trying to find a div that wraps a whole comment.
COMMENT_CONTAINER_SELECTOR = "div[class='xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs']" # From your script, likely unstable

# Relative XPaths to find elements *within* each COMMENT_CONTAINER_SELECTOR
# Example for username (likely outdated):
USERNAME_IN_COMMENT_XPATH = ".//span[contains(@class,'x193iq5w') or contains(@class,'xt0psk2')]/a[@role='link'] | .//a[@aria-label and @role='link' and not(contains(@href,'reaction')) and not(contains(@href,'comment_id'))][1]"
# Example for timestamp (likely outdated):
TIMESTAMP_IN_COMMENT_XPATH = ".//a[contains(@href,'comment_id=') and @role='link']//span[string-length(normalize-space(text())) > 0] | .//a[contains(@href,'comment_id=') and @role='link']"
# Example for comment text (this was your original selector, ensure it's relative or specific enough)
# If the COMMENT_CONTAINER_SELECTOR is good, this might be simpler like ".//div[@dir='auto']"
COMMENT_TEXT_IN_COMMENT_XPATH = ".//div[@class='xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs'] | .//div[@dir='auto' and not(.//a[@role='link'])]"
# --- End Selectors ---
USERNAME_IN_COMMENT_XPATH ="span[class='x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x676frb x1nxh6w3 x1sibtaa x1s688f xzsf02u']"
TIMESTAMP_IN_COMMENT_XPATH ="a[class='oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw']"

#TARGET_POST_URL = 'https://www.facebook.com/Cinexcepcion/posts/pfbid0bhBrBUN5ZMqWDJhk3nTJm9VzNspzcX9bcLSdxJNJBsu3UVZZuq9yjVVvTmjGcDMal'
TARGET_POST_URL1 = 'https://www.facebook.com/revolutapp/posts/pfbid036Ex3ZyiYbApCH3mNzt6e8KxtnnoX2Cs9GCkn4SWc8sarJPYJTvnNTRaXVJ8A7MvDl'
TARGET_POST_URL2 = 'https://www.facebook.com/share/1JF1EKDVpg/'
TARGET_POST_URL3 = 'https://fb.watch/zJFc4ifdn_/'
TARGET_POST_URL4 = 'https://www.facebook.com/reel/998981648315017'



DEFAULT_TIMEOUT = 35  # seconds for WebDriverWait

### Load environment variables
FACEBOOK_USER = os.getenv("facebook_user")
FACEBOOK_PASSWORD = os.getenv("facebook_password")

if not FACEBOOK_USER or not FACEBOOK_PASSWORD:
    print("Error: Facebook username or password not found in .env file or environment variables.")
    # Consider exiting or raising an error: raise ValueError("Missing Facebook credentials")

# %%
def initialize_driver(driver_path, binary_path):
    """Initializes and returns a Chrome WebDriver instance."""
    print("Initializing Chrome WebDriver...")
    service = Service(driver_path)
    options = webdriver.ChromeOptions()
    options.binary_location = binary_path
    # options.add_argument("--headless") # Optional: for running without a visible browser window
    options.add_argument("--disable-notifications")
    options.add_argument("--lang=en-US") # Helps with consistency
    # Suppress DevTools listening message
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    driver = webdriver.Chrome(service=service, options=options)
    print("WebDriver initialized.")
    return driver

def login_to_facebook(driver, wait, username, password):
    """Logs into Facebook."""
    print(f"Navigating to login page: {FACEBOOK_LOGIN_URL}")
    driver.maximize_window()
    driver.get(FACEBOOK_LOGIN_URL)

    print("Attempting to log in...")
    username_field = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, USERNAME_SELECTOR_LOGIN)))
    password_field = driver.find_element(By.CSS_SELECTOR, PASSWORD_SELECTOR_LOGIN)

    username_field.clear()
    password_field.clear()
    username_field.send_keys(username)
    password_field.send_keys(password)
    print("Facebook credentials entered.")

    login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, LOGIN_BUTTON_SELECTOR)))
    login_button.click()
    print("Login button clicked.")

    # Wait for login to complete - replace with a more reliable wait for a specific element on the home page
    print("Waiting for login to process...")
    try:
        # Example: Wait for the "Home" link or a search bar, indicating successful login.
        # Adjust selector as needed.
        wait.until(EC.presence_of_element_located((By.XPATH, "//a[@aria-label='Home'] | //input[@aria-label='Search Facebook']")))
        print("Login appears successful.")
    except TimeoutException:
        print("Timed out waiting for a known element after login. Login might have failed or page structure changed.")
        # Check for login failure messages or CAPTCHAs if this happens often.
        # For now, we'll proceed, but this is a point of potential failure.
        time.sleep(5) # Fallback sleep

# %%
def extract_post_comments_to_dataframe(
    driver: webdriver.Chrome,
    wait: WebDriverWait,
    post_url: str,
    container_selector: str,
    username_xpath: str,
    timestamp_xpath: str,
    comment_text_xpath: str,
    scroll_attempts: int = 5,
    scroll_delay: float = 2.5
) -> pd.DataFrame:
    """
    Navigates to a Facebook post, scrolls to load comments, extracts username,
    timestamp, and comment text, and returns them as a Pandas DataFrame.

    Args:
        driver: The Selenium WebDriver instance.
        wait: The Selenium WebDriverWait instance.
        post_url: The URL of the Facebook post.
        container_selector: CSS selector for the main container of a single comment.
        username_xpath: XPath relative to the container to find the username.
        timestamp_xpath: XPath relative to the container to find the timestamp.
        comment_text_xpath: XPath relative to the container to find the comment text.
        scroll_attempts: Number of times to scroll down to load more comments.
        scroll_delay: Delay in seconds between scrolls.

    Returns:
        A Pandas DataFrame with columns: 'username', 'timestamp', 'comment'.
    """
    print(f"Navigating directly to post: {post_url}")
    driver.get(post_url)
    print("Waiting for post page to load...")

    try:
        # Wait for a general post indicator.
        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@role='article'] | //div[contains(@aria-label, 'post')]")))
        print("Post page appears to be loading.")
    except TimeoutException:
        print("Timed out waiting for the main post content. Page structure might have changed or load is slow.")
        return pd.DataFrame(columns=['username', 'timestamp', 'comment']) # Return empty DataFrame

    # Scroll down to load comments
    print(f"Scrolling down {scroll_attempts} times to load comments...")
    for i in range(scroll_attempts):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        print(f"Scroll attempt {i+1}/{scroll_attempts}")
        time.sleep(scroll_delay) # Wait for comments to load

    extracted_comments_data = []
    print(f"Attempting to find comment containers using selector: {container_selector}")
    try:
        # Wait for at least one comment container to be present.
        comment_containers = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, container_selector)))
        print(f"Found {len(comment_containers)} potential comment containers.")
    except TimeoutException:
        print(f"Timed out waiting for comment containers with selector: {container_selector}")
        print("This could be due to: no comments, outdated selector, or page not loading comments.")
        return pd.DataFrame(columns=['username', 'timestamp', 'comment'])
    except NoSuchElementException: # Should be caught by TimeoutException with presence_of_all_elements_located
        print(f"No comment containers found with selector: {container_selector}.")
        return pd.DataFrame(columns=['username', 'timestamp', 'comment'])

    print("\n--- Extracting Comment Details ---")
    for i, container in enumerate(comment_containers):
        username = "N/A"
        timestamp = "N/A"
        comment_text = "N/A"

        try:
            # Find username within the container
            username_element = container.find_element(By.XPATH, username_xpath)
            username = username_element.text.strip()
            if not username: # Sometimes text is empty but in an attribute
                 username = username_element.get_attribute("aria-label") or username_element.get_attribute("textContent")
                 if username: username = username.strip()
        except NoSuchElementException:
            # print(f" - Username not found for comment {i+1}")
            pass # Silently pass if not found, default "N/A" will be used
        except Exception as e_user:
            print(f" - Error extracting username for comment {i+1}: {e_user}")


        try:
            # Find timestamp within the container
            timestamp_element = container.find_element(By.XPATH, timestamp_xpath)
            timestamp = timestamp_element.text.strip()
            if not timestamp: # Timestamps often in 'aria-label' or 'title' of a link
                timestamp = timestamp_element.get_attribute("aria-label") or timestamp_element.get_attribute("title") or timestamp_element.get_attribute("textContent")
                if timestamp: timestamp = timestamp.strip()
        except NoSuchElementException:
            # print(f" - Timestamp not found for comment {i+1}")
            pass
        except Exception as e_time:
            print(f" - Error extracting timestamp for comment {i+1}: {e_time}")

        try:
            # Find comment text within the container
            comment_text_element = container.find_element(By.XPATH, comment_text_xpath)
            comment_text = comment_text_element.text.strip()
        except NoSuchElementException:
            # print(f" - Comment text not found for comment {i+1}")
            pass
        except Exception as e_text:
            print(f" - Error extracting comment text for comment {i+1}: {e_text}")
        
        if username == "N/A" and timestamp == "N/A" and comment_text == "N/A":
            # If the container was found but no sub-elements, it might be an ad or other non-comment block
            # Or the sub-selectors are wrong.
            print(f"Comment {i+1}: Could not extract any specific details (username, timestamp, text). Container text: '{container.text[:100].strip() if container.text else 'EMPTY'}'")
            continue # Skip if nothing useful was extracted

        extracted_comments_data.append({
            'username': username,
            'timestamp': timestamp,
            'comment': comment_text
        })
        print(f"Comment {i+1}: User: '{username}', Time: '{timestamp}', Text: '{comment_text[:50]}...'")

    print("-----------------------------")
    df = pd.DataFrame(extracted_comments_data)
    return df



In [53]:
# %%
### Main script execution
'''
driver = None
try:
    if not FACEBOOK_USER or not FACEBOOK_PASSWORD:
        raise ValueError("Facebook credentials are not set. Please check your .env file.")

    driver = initialize_driver(CHROME_DRIVER_PATH, CHROME_BINARY_PATH)
    wait = WebDriverWait(driver, DEFAULT_TIMEOUT)

    login_to_facebook(driver, wait, FACEBOOK_USER, FACEBOOK_PASSWORD)

    # --- Extract comments from the target post ---
    print(f"\nStarting comment extraction for post: {TARGET_POST_URL}")
    comments_df = extract_post_comments_to_dataframe(
        driver,
        wait,
        TARGET_POST_URL2,
        COMMENT_CONTAINER_SELECTOR,
        USERNAME_IN_COMMENT_XPATH,
        TIMESTAMP_IN_COMMENT_XPATH,
        COMMENT_TEXT_IN_COMMENT_XPATH,
        scroll_attempts=5, # Adjust as needed
        scroll_delay=3.0
    )

    if not comments_df.empty:
        print("\n--- Successfully Extracted Comments DataFrame ---")
        print(comments_df.head())
        # You can now save the DataFrame to a file, e.g., CSV
        # comments_df.to_csv("facebook_comments.csv", index=False)
        # print("\nDataFrame saved to facebook_comments.csv")
    else:
        print("\nNo comments were extracted, or an error occurred during extraction.")

except ValueError as ve:
    print(f"Configuration Error: {ve}")
except TimeoutException as e:
    print(f"A critical timeout occurred during the process: {e}")
except NoSuchElementException as e:
    print(f"A required element was not found (check selectors and page structure): {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    import traceback
    traceback.print_exc()
finally:
    if driver:
        print("\nClosing the WebDriver.")
        # time.sleep(10) # Optional: keep browser open for a bit for debugging
        driver.quit()
    print("Script finished.")

'''

'\ndriver = None\ntry:\n    if not FACEBOOK_USER or not FACEBOOK_PASSWORD:\n        raise ValueError("Facebook credentials are not set. Please check your .env file.")\n\n    driver = initialize_driver(CHROME_DRIVER_PATH, CHROME_BINARY_PATH)\n    wait = WebDriverWait(driver, DEFAULT_TIMEOUT)\n\n    login_to_facebook(driver, wait, FACEBOOK_USER, FACEBOOK_PASSWORD)\n\n    # --- Extract comments from the target post ---\n    print(f"\nStarting comment extraction for post: {TARGET_POST_URL}")\n    comments_df = extract_post_comments_to_dataframe(\n        driver,\n        wait,\n        TARGET_POST_URL2,\n        COMMENT_CONTAINER_SELECTOR,\n        USERNAME_IN_COMMENT_XPATH,\n        TIMESTAMP_IN_COMMENT_XPATH,\n        COMMENT_TEXT_IN_COMMENT_XPATH,\n        scroll_attempts=5, # Adjust as needed\n        scroll_delay=3.0\n    )\n\n    if not comments_df.empty:\n        print("\n--- Successfully Extracted Comments DataFrame ---")\n        print(comments_df.head())\n        # You can now sa

In [54]:
#comments_df

## Análisis principal divisas

In [55]:
'''
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

print("Attempting to start Chrome...")
try:
    options = webdriver.ChromeOptions()
    options.binary_location = r'E:/Users/1167486/Local/Drivers_web/chrome-win64/chrome.exe'
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--no-sandbox') # Often helpful in restricted environments
    options.add_argument('--disable-dev-shm-usage') # Overcomes resource limits
    options.add_argument('--headless') # Try headless to rule out display/GUI issues
    options.add_argument('--disable-gpu') # Often needed with headless

    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    print("Driver started. Navigating to Google...")
    driver.get("https://www.google.com")
    print(f"Page title: {driver.title}")
    driver.quit()
    print("Driver quit successfully.")
except Exception as e:
    print(f"An error occurred: {e}")
'''

'\nfrom selenium import webdriver\nfrom selenium.webdriver.chrome.service import Service as ChromeService\nfrom webdriver_manager.chrome import ChromeDriverManager\n\nprint("Attempting to start Chrome...")\ntry:\n    options = webdriver.ChromeOptions()\n    options.binary_location = r\'E:/Users/1167486/Local/Drivers_web/chrome-win64/chrome.exe\'\n    options.add_argument(\'--disable-blink-features=AutomationControlled\')\n    options.add_argument(\'--no-sandbox\') # Often helpful in restricted environments\n    options.add_argument(\'--disable-dev-shm-usage\') # Overcomes resource limits\n    options.add_argument(\'--headless\') # Try headless to rule out display/GUI issues\n    options.add_argument(\'--disable-gpu\') # Often needed with headless\n\n    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)\n    print("Driver started. Navigating to Google...")\n    driver.get("https://www.google.com")\n    print(f"Page title: {driver.title}")

In [56]:
'''
options = webdriver.ChromeOptions()
options.binary_location = r'E:/Users/1167486/Local/Drivers_web/chrome-win64/chrome.exe'
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage') 
#options.add_argument('--headless') 
#options.add_argument('--disable-gpu') 
driverFB = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), 
                          options=options)
'''

"\noptions = webdriver.ChromeOptions()\noptions.binary_location = r'E:/Users/1167486/Local/Drivers_web/chrome-win64/chrome.exe'\noptions.add_argument('--disable-blink-features=AutomationControlled')\noptions.add_argument('--no-sandbox')\noptions.add_argument('--disable-dev-shm-usage') \n#options.add_argument('--headless') \n#options.add_argument('--disable-gpu') \ndriverFB = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), \n                          options=options)\n"

In [57]:
'''
FACEBOOK_LOGIN_URL = 'https://www.facebook.com/login/'
driverFB.maximize_window()
driverFB.get(FACEBOOK_LOGIN_URL)
time.sleep(5)
'''

"\nFACEBOOK_LOGIN_URL = 'https://www.facebook.com/login/'\ndriverFB.maximize_window()\ndriverFB.get(FACEBOOK_LOGIN_URL)\ntime.sleep(5)\n"

In [58]:
'''
wait = WebDriverWait(driver, 10)


try:
    print("Attempting to log in...")
    username_field = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, USERNAME_SELECTOR_LOGIN)))
    password_field = driver.find_element(By.CSS_SELECTOR, PASSWORD_SELECTOR_LOGIN)

    username_field.clear()
    password_field.clear()
    username_field.send_keys(FACEBOOK_USER)
    password_field.send_keys(FACEBOOK_PASSWORD)
    print("Facebook credentials entered.")

    login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, LOGIN_BUTTON_SELECTOR)))
    login_button.click()
    print("Login button clicked.")

    # Wait for login to complete - replace with a more reliable wait for a specific element on the home page
    print("Waiting for login to process...")
    try:
        # Example: Wait for the "Home" link or a search bar, indicating successful login.
        # Adjust selector as needed.
        wait.until(EC.presence_of_element_located((By.XPATH, "//a[@aria-label='Home'] | //input[@aria-label='Search Facebook']")))
        print("Login appears successful.")
    except TimeoutException:
        print("Timed out waiting for a known element after login. Login might have failed or page structure changed.")
        # Check for login failure messages or CAPTCHAs if this happens often.
        # For now, we'll proceed, but this is a point of potential failure.
        time.sleep(5) # Fallback sleep

except Exception as e:
    print(f"An error occurred during login steps: {e}")
    # Depending on your script, you might want to quit the driver here
    driverFB.quit()
'''

'\nwait = WebDriverWait(driver, 10)\n\n\ntry:\n    print("Attempting to log in...")\n    username_field = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, USERNAME_SELECTOR_LOGIN)))\n    password_field = driver.find_element(By.CSS_SELECTOR, PASSWORD_SELECTOR_LOGIN)\n\n    username_field.clear()\n    password_field.clear()\n    username_field.send_keys(FACEBOOK_USER)\n    password_field.send_keys(FACEBOOK_PASSWORD)\n    print("Facebook credentials entered.")\n\n    login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, LOGIN_BUTTON_SELECTOR)))\n    login_button.click()\n    print("Login button clicked.")\n\n    # Wait for login to complete - replace with a more reliable wait for a specific element on the home page\n    print("Waiting for login to process...")\n    try:\n        # Example: Wait for the "Home" link or a search bar, indicating successful login.\n        # Adjust selector as needed.\n        wait.until(EC.presence_of_element_located((By.XPATH, "

In [80]:
import pandas as pd 
import random
random.seed(42)
times = [20, 30, 35, 15, 25, 7, 10, 12, 5, 8, 18, 22, 28, 40]


In [81]:
mainPath = 'E:/Users/1167486/Local/proyectos/divisas/Scraper_divisas'

FbLinks = pd.read_csv(mainPath + '/Links_facebook_to_scrape.csv')

FbLinks = FbLinks[FbLinks['link'].str.contains('share')]

In [82]:
test = FbLinks.iloc[0:62]
test

,link,app,social_media
29,https://www.facebook.com/share/v/16cpnzsCdZ/,DollarApp,Facebook
30,https://www.facebook.com/share/r/1Bsagcmmmt/,DollarApp,Facebook
31,https://www.facebook.com/share/r/1HHhhWg1N6/,DollarApp,Facebook
32,https://www.facebook.com/share/1C47ivC3PV/,DollarApp,Facebook
33,https://www.facebook.com/share/1JF1EKDVpg/,DollarApp,Facebook
...,...,...,...
86,https://www.facebook.com/share/p/1BHafqM44V/,Wise,Facebook
87,https://www.facebook.com/share/p/16oxUy5ait/,Wise,Facebook
88,https://www.facebook.com/share/p/1BjydL5k11/,Wise,Facebook
89,https://www.facebook.com/share/p/1ABjYPprhf/,Wise,Facebook


In [ ]:
'''
listDf = []
for index, row in test.iterrows():
#for index, row in FbLinks.iterrows():
    print(f'index: {index + 1}')
    print(row['link'])
    print(row['app'])
    x = random.sample(times, len(times))
    print(f'random time: {x[0]}')
    print('-------------------')
    #time.sleep(x[0])
    time.sleep(10)
    if row['app']!='Wise':
        scroll_attempts = 15
    else: 
        scroll_attempts = 10 
    #print(f'Results required: {scroll_attempts}')
    #print(f'max scroll attempts: {scroll_attempts}')
    #wait = WebDriverWait(driver, x)


    # --- Extract comments from the target post ---
    print(f"\nStarting comment extraction for post: {row['link']}")
    postUrl = row['link']
    facebookDfScrape = extract_post_comments_to_dataframe(
        driverFB,
        wait,
        postUrl,
        COMMENT_CONTAINER_SELECTOR,
        USERNAME_IN_COMMENT_XPATH,
        TIMESTAMP_IN_COMMENT_XPATH,
        COMMENT_TEXT_IN_COMMENT_XPATH,
        scroll_attempts=scroll_attempts,
        scroll_delay=3.0
    )


    facebookDfScrape['app'] = row['app']
    facebookDfScrape['original_tweet_link'] = row['link']
    facebookDfScrape['original_social_media'] = row['social_media']
    listDf.append(facebookDfScrape)
'''

index: 30
https://www.facebook.com/share/v/16cpnzsCdZ/
DollarApp
random time: 8
-------------------

Starting comment extraction for post: https://www.facebook.com/share/v/16cpnzsCdZ/
Navigating directly to post: https://www.facebook.com/share/v/16cpnzsCdZ/
Waiting for post page to load...
Post page appears to be loading.
Scrolling down 15 times to load comments...
Scroll attempt 1/15
Scroll attempt 2/15
Scroll attempt 3/15
Scroll attempt 4/15
Scroll attempt 5/15
Scroll attempt 6/15
Scroll attempt 7/15
Scroll attempt 8/15
Scroll attempt 9/15
Scroll attempt 10/15
Scroll attempt 11/15
Scroll attempt 12/15
Scroll attempt 13/15
Scroll attempt 14/15
Scroll attempt 15/15
Attempting to find comment containers using selector: div[aria-label*='Comment by'], div.x78zum5.x1n2onr6.xh8yej3
Found 5 potential comment containers.

--- Extracting Comment Details ---
Comment 1: Could not extract any specific details (username, timestamp, text). Container text: 'EMPTY'
Comment 2: Could not extract any sp

In [87]:
# %%
### Main script execution
listDf = []
for index, row in test.iterrows():
    print(f'index: {index + 1}')
    x = random.sample(times, len(times))
    print(f'random time: {x[0]}')
    print('-------------------')
    time.sleep(x[0])
    
    driver = None
    try:
        if not FACEBOOK_USER or not FACEBOOK_PASSWORD:
            raise ValueError("Facebook credentials are not set. Please check your .env file.")

        driver = initialize_driver(CHROME_DRIVER_PATH, CHROME_BINARY_PATH)
        wait = WebDriverWait(driver, DEFAULT_TIMEOUT)

        login_to_facebook(driver, wait, FACEBOOK_USER, FACEBOOK_PASSWORD)
        TARGET_POST_URL = row['link']
        # --- Extract comments from the target post ---
        print(f"\nStarting comment extraction for post: {TARGET_POST_URL}")
        comments_df = extract_post_comments_to_dataframe(
            driver,
            wait,
            TARGET_POST_URL,
            COMMENT_CONTAINER_SELECTOR,
            USERNAME_IN_COMMENT_XPATH,
            TIMESTAMP_IN_COMMENT_XPATH,
            COMMENT_TEXT_IN_COMMENT_XPATH,
            scroll_attempts=15, 
            scroll_delay=3.0
        )

        if not comments_df.empty:
            print("\n--- Successfully Extracted Comments DataFrame ---")
            print(comments_df.head())
            comments_df['app'] = row['app']
            comments_df['original_facebook_link'] = row['link']
            comments_df['original_social_media'] = row['social_media']
            listDf.append(comments_df)

            # You can now save the DataFrame to a file, e.g., CSV
            # comments_df.to_csv("facebook_comments.csv", index=False)
            # print("\nDataFrame saved to facebook_comments.csv")
        else:
            print("\nNo comments were extracted, or an error occurred during extraction.")

    except ValueError as ve:
        print(f"Configuration Error: {ve}")
    except TimeoutException as e:
        print(f"A critical timeout occurred during the process: {e}")
    except NoSuchElementException as e:
        print(f"A required element was not found (check selectors and page structure): {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()
    finally:
        if driver:
            print("\nClosing the WebDriver.")
            # time.sleep(10) # Optional: keep browser open for a bit for debugging
            driver.quit()
        print("Script finished.")



index: 30
random time: 5
-------------------
Initializing Chrome WebDriver...
WebDriver initialized.
Navigating to login page: https://www.facebook.com/login/
Attempting to log in...
Facebook credentials entered.
Login button clicked.
Waiting for login to process...
Timed out waiting for a known element after login. Login might have failed or page structure changed.

Starting comment extraction for post: https://www.facebook.com/share/v/16cpnzsCdZ/
Navigating directly to post: https://www.facebook.com/share/v/16cpnzsCdZ/
Waiting for post page to load...
Post page appears to be loading.
Scrolling down 15 times to load comments...
Scroll attempt 1/15
Scroll attempt 2/15
Scroll attempt 3/15
Scroll attempt 4/15
Scroll attempt 5/15
Scroll attempt 6/15
Scroll attempt 7/15
Scroll attempt 8/15
Scroll attempt 9/15
Scroll attempt 10/15
Scroll attempt 11/15
Scroll attempt 12/15
Scroll attempt 13/15
Scroll attempt 14/15
Scroll attempt 15/15
Attempting to find comment containers using selector: div

Traceback (most recent call last):
  File "e:\Users\1167486\AppData\Local\anaconda3\envs\Iris_WebScrapper\Lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "e:\Users\1167486\AppData\Local\anaconda3\envs\Iris_WebScrapper\Lib\site-packages\urllib3\connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "e:\Users\1167486\AppData\Local\anaconda3\envs\Iris_WebScrapper\Lib\http\client.py", line 1430, in getresponse
    response.begin()
  File "e:\Users\1167486\AppData\Local\anaconda3\envs\Iris_WebScrapper\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "e:\Users\1167486\AppData\Local\anaconda3\envs\Iris_WebScrapper\Lib\http\client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-

Script finished.
index: 55
random time: 25
-------------------
Initializing Chrome WebDriver...
WebDriver initialized.
Navigating to login page: https://www.facebook.com/login/
Attempting to log in...
Facebook credentials entered.
Login button clicked.
Waiting for login to process...
Timed out waiting for a known element after login. Login might have failed or page structure changed.

Starting comment extraction for post: https://www.facebook.com/share/p/1Ftzv1P98U/
Navigating directly to post: https://www.facebook.com/share/p/1Ftzv1P98U/
Waiting for post page to load...
Post page appears to be loading.
Scrolling down 15 times to load comments...
Scroll attempt 1/15
Scroll attempt 2/15
Scroll attempt 3/15
Scroll attempt 4/15
Scroll attempt 5/15
Scroll attempt 6/15
Scroll attempt 7/15
Scroll attempt 8/15
Scroll attempt 9/15
Scroll attempt 10/15
Scroll attempt 11/15
Scroll attempt 12/15
Scroll attempt 13/15
Scroll attempt 14/15
Scroll attempt 15/15
Attempting to find comment containers u

Traceback (most recent call last):
  File "e:\Users\1167486\AppData\Local\anaconda3\envs\Iris_WebScrapper\Lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "e:\Users\1167486\AppData\Local\anaconda3\envs\Iris_WebScrapper\Lib\site-packages\urllib3\connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "e:\Users\1167486\AppData\Local\anaconda3\envs\Iris_WebScrapper\Lib\http\client.py", line 1430, in getresponse
    response.begin()
  File "e:\Users\1167486\AppData\Local\anaconda3\envs\Iris_WebScrapper\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "e:\Users\1167486\AppData\Local\anaconda3\envs\Iris_WebScrapper\Lib\http\client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-

Script finished.
index: 61
random time: 25
-------------------
Initializing Chrome WebDriver...
WebDriver initialized.
Navigating to login page: https://www.facebook.com/login/
Attempting to log in...
Facebook credentials entered.
Login button clicked.
Waiting for login to process...
Timed out waiting for a known element after login. Login might have failed or page structure changed.

Starting comment extraction for post: https://www.facebook.com/share/p/16Hbc2dap3/
Navigating directly to post: https://www.facebook.com/share/p/16Hbc2dap3/
Waiting for post page to load...
Post page appears to be loading.
Scrolling down 15 times to load comments...
Scroll attempt 1/15
Scroll attempt 2/15
Scroll attempt 3/15
Scroll attempt 4/15
Scroll attempt 5/15
Scroll attempt 6/15
Scroll attempt 7/15
Scroll attempt 8/15
Scroll attempt 9/15
Scroll attempt 10/15
Scroll attempt 11/15
Scroll attempt 12/15
Scroll attempt 13/15
Scroll attempt 14/15
Scroll attempt 15/15
Attempting to find comment containers u

In [88]:
facebookDivisas = pd.concat(listDf, ignore_index=True)

In [89]:
facebookDivisas.shape

(549, 6)

In [90]:
facebookDivisas.head(20)

,username,timestamp,comment,app,original_facebook_link,original_social_media
0,N/A,N/A,Si estás cansado de los cálculos interminables...,DollarApp,https://www.facebook.com/share/v/16cpnzsCdZ/,Facebook
1,N/A,N/A,the card is available for Colombia ?,DollarApp,https://www.facebook.com/share/v/16cpnzsCdZ/,Facebook
2,N/A,N/A,No me llega el pago que envié,DollarApp,https://www.facebook.com/share/v/16cpnzsCdZ/,Facebook
3,N/A,N/A,"Hola Gustavo, tuvimos un problema ayer por la ...",DollarApp,https://www.facebook.com/share/1C47ivC3PV/,Facebook
4,N/A,N/A,Habiliten que se puedan usar los datos de cuen...,DollarApp,https://www.facebook.com/share/1C47ivC3PV/,Facebook
5,N/A,N/A,Rodrigo Zelaya,DollarApp,https://www.facebook.com/share/1C47ivC3PV/,Facebook
6,N/A,N/A,Hola DolarApp esta es una tarjeta de crédito?,DollarApp,https://www.facebook.com/share/1C47ivC3PV/,Facebook
7,N/A,N/A,¿Es posible depositar cheques del U.S. Treasur...,DollarApp,https://www.facebook.com/share/1C47ivC3PV/,Facebook
8,N/A,N/A,Is it working in Brazil?,DollarApp,https://www.facebook.com/share/1C47ivC3PV/,Facebook
9,N/A,N/A,Se puede hacer transferencia a países del sur ...,DollarApp,https://www.facebook.com/share/1C47ivC3PV/,Facebook


In [91]:
facebookDivisas.drop(['username', 'timestamp'], inplace=True, axis=1)

In [92]:
facebookDivisas.head(20)

,comment,app,original_facebook_link,original_social_media
0,Si estás cansado de los cálculos interminables...,DollarApp,https://www.facebook.com/share/v/16cpnzsCdZ/,Facebook
1,the card is available for Colombia ?,DollarApp,https://www.facebook.com/share/v/16cpnzsCdZ/,Facebook
2,No me llega el pago que envié,DollarApp,https://www.facebook.com/share/v/16cpnzsCdZ/,Facebook
3,"Hola Gustavo, tuvimos un problema ayer por la ...",DollarApp,https://www.facebook.com/share/1C47ivC3PV/,Facebook
4,Habiliten que se puedan usar los datos de cuen...,DollarApp,https://www.facebook.com/share/1C47ivC3PV/,Facebook
5,Rodrigo Zelaya,DollarApp,https://www.facebook.com/share/1C47ivC3PV/,Facebook
6,Hola DolarApp esta es una tarjeta de crédito?,DollarApp,https://www.facebook.com/share/1C47ivC3PV/,Facebook
7,¿Es posible depositar cheques del U.S. Treasur...,DollarApp,https://www.facebook.com/share/1C47ivC3PV/,Facebook
8,Is it working in Brazil?,DollarApp,https://www.facebook.com/share/1C47ivC3PV/,Facebook
9,Se puede hacer transferencia a países del sur ...,DollarApp,https://www.facebook.com/share/1C47ivC3PV/,Facebook


In [93]:
facebookDivisas['original_facebook_link'].unique() 

array(['https://www.facebook.com/share/v/16cpnzsCdZ/',
       'https://www.facebook.com/share/1C47ivC3PV/',
       'https://www.facebook.com/share/1JF1EKDVpg/',
       'https://www.facebook.com/share/p/1CUsEYixnT/',
       'https://www.facebook.com/share/p/1ZmfiV6XYo/',
       'https://www.facebook.com/share/p/1Ag4WMGvou/',
       'https://www.facebook.com/share/p/1EiVAqzwLK/',
       'https://www.facebook.com/share/p/19kP9MkXyQ/',
       'https://www.facebook.com/share/p/1ArCbYHKzV/',
       'https://www.facebook.com/share/p/1NUkWtznao/',
       'https://www.facebook.com/share/p/16RWSBvqzk/',
       'https://www.facebook.com/share/p/18hbXhWHk6/',
       'https://www.facebook.com/share/p/18qP2QB1XA/',
       'https://www.facebook.com/share/p/18tv1kEMjb/',
       'https://www.facebook.com/share/p/16idGNMw6C/',
       'https://www.facebook.com/share/p/1Z2n7VRVUV/',
       'https://www.facebook.com/share/p/1HX7LJgwu6/',
       'https://www.facebook.com/share/p/1H62k1Mhmc/',
       'https:

In [94]:
newFilePath = mainPath + '/facebook_scraped_data_divisas.csv'

In [95]:
facebookDivisas.to_csv(newFilePath, index=False)

## Test Zone

In [19]:
load_dotenv()

True

In [8]:
### Load environment variables
### Now I'm using the .env file to load the variables
FACEBOOK_USER = os.getenv("facebook_user")
FACEBOOK_PASSWORD = os.getenv("facebook_password")
## Loading the service and options for the chrome driver
service = Service('E:/Users/1167486/Local/Drivers_web/chromedriver-win64/chromedriver.exe')
options = webdriver.ChromeOptions()
options.binary_location = r'E:/Users/1167486/Local/Drivers_web/chrome-win64/chrome.exe'

driver = webdriver.Chrome(service=service, options=options)


In [37]:
driver.maximize_window()
driver.get('https://www.facebook.com/login/')


In [38]:
ssl._create_default_https_context = ssl._create_unverified_context
textos = []


In [ ]:
time.sleep(3)
username = driver.find_element("css selector", "input[name='email']")
password = driver.find_element("css selector", "input[name='pass']")

In [40]:
username.clear()
password.clear()


In [41]:
username.send_keys(FACEBOOK_USER)
password.send_keys(FACEBOOK_PASSWORD)


In [42]:
login = driver.find_element("css selector", "button[type='submit']").click()
time.sleep(6)


In [43]:
name = 'cinexcepción'
driver.get("https://www.facebook.com/search/pages/?q="+name)
time.sleep(2)


In [44]:
post = 'https://www.facebook.com/Cinexcepcion/posts/pfbid0bhBrBUN5ZMqWDJhk3nTJm9VzNspzcX9bcLSdxJNJBsu3UVZZuq9yjVVvTmjGcDMal'

driver.get(post)
time.sleep(2)




In [34]:
#comments = driver.find_elements("css selector", "div[class='x1lliihq x1ja2u2z xhkezso x1gmr53x x13faqbe x19nzlky x1f6m7m3 x13qkpa x1ur3af0 x1544l6x x18d9i69 xw4jnvo']")
comments = driver.find_elements("css selector", "div[class='xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs']")


[<selenium.webdriver.remote.webelement.WebElement (session="0ae2fa35bc42ab9c4eaee5adb1efc123", element="f.BE59274331F024596A43C7306F8B2F98.d.1DC55366E611F29EDE6C7C9D745D838A.e.7960")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0ae2fa35bc42ab9c4eaee5adb1efc123", element="f.BE59274331F024596A43C7306F8B2F98.d.1DC55366E611F29EDE6C7C9D745D838A.e.8075")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0ae2fa35bc42ab9c4eaee5adb1efc123", element="f.BE59274331F024596A43C7306F8B2F98.d.1DC55366E611F29EDE6C7C9D745D838A.e.8183")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0ae2fa35bc42ab9c4eaee5adb1efc123", element="f.BE59274331F024596A43C7306F8B2F98.d.1DC55366E611F29EDE6C7C9D745D838A.e.8294")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0ae2fa35bc42ab9c4eaee5adb1efc123", element="f.BE59274331F024596A43C7306F8B2F98.d.1DC55366E611F29EDE6C7C9D745D838A.e.8394")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0ae2fa35bc42ab9

In [46]:
comments_text_list = []

for comment in comments:
    print(comment.text)
    #comments_text_list.append(comment.text)
 


InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00007FF68B813C25+77845]
	GetHandleVerifier [0x00007FF68B813C80+77936]
	(No symbol) [0x00007FF68B5C8F6C]
	(No symbol) [0x00007FF68B60F6FF]
	(No symbol) [0x00007FF68B647152]
	(No symbol) [0x00007FF68B641B82]
	(No symbol) [0x00007FF68B640C49]
	(No symbol) [0x00007FF68B595A75]
	GetHandleVerifier [0x00007FF68BAC0C5D+2883661]
	GetHandleVerifier [0x00007FF68BABB55E+2861390]
	GetHandleVerifier [0x00007FF68BAD8A43+2981427]
	GetHandleVerifier [0x00007FF68B82ECFA+188650]
	GetHandleVerifier [0x00007FF68B836F6F+222047]
	(No symbol) [0x00007FF68B594AE8]
	GetHandleVerifier [0x00007FF68BBC5DA8+3953048]
	BaseThreadInitThunk [0x00007FFD6DD97614+20]
	RtlUserThreadStart [0x00007FFD6FA026F1+33]


## Test Zone 2

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import random
import json


In [7]:
class FacebookScraper:
    def __init__(self, email, password):
        self.email = email
        self.password = password
        self.driver = None
        
    def initialize_driver(self):
        """Initialize the Edge webdriver with custom options"""
        options = webdriver.EdgeOptions()
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option("useAutomationExtension", False)
        
        self.driver = webdriver.Edge(options=options)
        self.driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        
    def simulate_human_typing(self, element, text):
        """Simulate human-like typing patterns"""
        for char in text:
            element.send_keys(char)
            time.sleep(random.uniform(0.1, 0.3))
            if random.random() < 0.1:
                time.sleep(random.uniform(0.3, 0.7))
                
    def login(self):
        """Login to Facebook"""
        self.driver.get("https://www.facebook.com/login")
        
        # Enter email
        email_input = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.NAME, "email"))
        )
        self.simulate_human_typing(email_input, self.email)
        
        # Enter password
        password_input = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.NAME, "pass"))
        )
        self.simulate_human_typing(password_input, self.password)
        
        # Click login button
        login_button = self.driver.find_element(By.XPATH, "//button[@type='submit']")
        ActionChains(self.driver)\
            .move_to_element(login_button)\
            .pause(random.uniform(0.2, 0.4))\
            .click()\
            .perform()
            
        time.sleep(15)
        
    def navigate_to_profile(self, profile_url):
        """Navigate to a specific Facebook profile"""
        self.driver.get(profile_url)
        time.sleep(4)
        
    def slow_scroll(self, step=500):
        """Scroll the page slowly"""
        self.driver.execute_script(f"window.scrollBy(0, {step});")
        time.sleep(2)
        
    def extract_posts_with_bs(self):
        """Extract posts data using BeautifulSoup"""
        page_source = self.driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        posts_data = []
        
        posts = soup.find_all("div", {"class": "x1n2onr6 x1ja2u2z"})
        
        for post in posts:
            try:
                message_elements = post.find_all("div", {"data-ad-preview": "message"})
                post_text = " ".join([msg.get_text(strip=True) for msg in message_elements])
                
                likes_element = post.select_one("span.xt0b8zv.x1jx94hy.xrbpyxo.xl423tq > span > span")
                likes = likes_element.get_text(strip=True) if likes_element else None
                
                comments_element = post.select("div > div > span > div > div > div > span > span.html-span ")
                comments = comments_element[0].text if comments_element else None
                
                
                shares_element =post.select("div > div > span > div > div > div > span > span.html-span ")
                shares = shares_element[1].text if shares_element else None

                timeelement=post.select_one("div.xu06os2.x1ok221b > span > div > span > span > a > span")
                post_time= timeelement.get_text(strip=True) if timeelement else None

                
                posts_data.append({
                    "post_text": post_text,
                    "likes": likes,
                    "comments": comments,
                    "shares": shares,
                    "post_time": post_time
                })
            except Exception as e:
                print("Error extracting post data:", e)
                
        return posts_data
        
    def remove_duplicates(self, data_list):
        """Remove duplicate posts"""
        seen = set()
        unique_data = []
        for data in data_list:
            data_tuple = tuple(data.items())
            if data_tuple not in seen:
                seen.add(data_tuple)
                unique_data.append(data)
        return unique_data
        
    def scrape_posts(self, max_posts):
        """Scrape a specified number of posts"""
        all_posts = []
        
        while len(all_posts) < max_posts:
            posts = self.extract_posts_with_bs()
            all_posts.extend(posts)
            all_posts = self.remove_duplicates(all_posts)
            print(f"Extracted {len(all_posts)} unique posts so far.")
            # print(all_posts)
            self.slow_scroll()
            
            if len(all_posts) >= max_posts:
                break
                
        return all_posts[:max_posts]



    def print_posts(self, posts_data):
        """Print the scraped posts data"""
        for idx, post in enumerate(posts_data, start=1):
            print(f"Post {idx}:")
            print(f"Text: {post['post_text']}")
            print(f"Likes: {post['likes']}")
            print(f"Comments: {post['comments']}")
            print(f"Shares: {post['shares']}")
            print(f"Time Posted: {post['post_time']}")
            print("-" * 50)
            
    def close(self):
        """Close the browser"""
        if self.driver:
            self.driver.quit()



In [8]:
# Example usage
if __name__ == "__main__":
    # Initialize the scraper
    scraper = FacebookScraper(FACEBOOK_USER, FACEBOOK_PASSWORD)
    
    try:
        # Setup and login
        scraper.initialize_driver()
        scraper.login()
        
        # Navigate to Cristiano Ronaldo's profile
        scraper.navigate_to_profile("https://www.facebook.com/cnn")
        
        # Scrape 10 posts
        posts_data = scraper.scrape_posts(max_posts=10)
        
        # Print the results
        scraper.print_posts(posts_data)
        
    finally:
        # Clean up
        scraper.close()


Extracted 3 unique posts so far.
Extracted 3 unique posts so far.
Extracted 3 unique posts so far.
Extracted 3 unique posts so far.
Extracted 3 unique posts so far.
Extracted 3 unique posts so far.
Extracted 3 unique posts so far.
Extracted 3 unique posts so far.
Extracted 3 unique posts so far.
Extracted 3 unique posts so far.
Extracted 3 unique posts so far.
Extracted 3 unique posts so far.
Extracted 3 unique posts so far.
Extracted 3 unique posts so far.
Extracted 3 unique posts so far.
Extracted 3 unique posts so far.
Extracted 3 unique posts so far.
Extracted 3 unique posts so far.


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: MicrosoftEdge=136.0.3240.76)
Stacktrace:
	GetHandleVerifier [0x00007FF639E3F965+25029]
	(No symbol) [0x00007FF639D94970]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF63A09AA6A+1947706]
	(No symbol) [0x00007FF639B4B790]
	(No symbol) [0x00007FF639B6A10A]
	(No symbol) [0x00007FF639BCED27]
	(No symbol) [0x00007FF639BE609A]
	(No symbol) [0x00007FF639BC8DF3]
	(No symbol) [0x00007FF639B9D6A6]
	(No symbol) [0x00007FF639B9CBB3]
	(No symbol) [0x00007FF639B9D4D3]
	(No symbol) [0x00007FF639CA5F0D]
	(No symbol) [0x00007FF639CB3AAF]
	(No symbol) [0x00007FF639CAC49F]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF639F04A1A+284650]
	(No symbol) [0x00007FF639DA2301]
	(No symbol) [0x00007FF639D9ADA4]
	(No symbol) [0x00007FF639D9AEF3]
	(No symbol) [0x00007FF639D8CB36]
	BaseThreadInitThun

In [ ]:
posts_data

[{'post_text': '',
  'likes': '76',
  'comments': '16',
  'shares': '18',
  'post_time': None},
 {'post_text': '',
  'likes': '74',
  'comments': '36',
  'shares': '8',
  'post_time': None},
 {'post_text': '',
  'likes': '650',
  'comments': '506',
  'shares': '78',
  'post_time': None},
 {'post_text': '',
  'likes': None,
  'comments': None,
  'shares': None,
  'post_time': None},
 {'post_text': '',
  'likes': '292',
  'comments': None,
  'shares': None,
  'post_time': None},
 {'post_text': '',
  'likes': '233',
  'comments': None,
  'shares': None,
  'post_time': None},
 {'post_text': '',
  'likes': '167',
  'comments': '105',
  'shares': '17',
  'post_time': None},
 {'post_text': '',
  'likes': '1,4\xa0mil',
  'comments': '1\xa0mil',
  'shares': '179',
  'post_time': None},
 {'post_text': '',
  'likes': '5,3\xa0mil',
  'comments': '724',
  'shares': '499',
  'post_time': None},
 {'post_text': '',
  'likes': '292',
  'comments': '196',
  'shares': '89',
  'post_time': None}]

## Test Zone 3 

In [12]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import numpy as np
import json

from selenium.webdriver.common.by import By

import time
from io import StringIO
import os

def login_facebook(username, password, driver):
    '''Log in to Facebook'''
    try:
        # Wait 10s until the login form is loaded
        username_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "email")))
        username_field.send_keys(username)

        password_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "pass")))
        password_field.send_keys(password)

        # Submit log in form 
        password_field.send_keys(Keys.RETURN)

    except Exception as e:
        print("Logged in failed. \n")

    finally:
        print("Logged in successfully")
        
def configure_driver():
    '''Configure the webdriver'''

    #Configurations
    #webdriver_path = "C:\\Program Files\\Scrapper1\\chromedriver.exe" #your webdriver path (chromedriver.exe)
    webdriver_path = "E:/Users/1167486/Local/Drivers_web/chromedriver-win64/chromedriver.exe" #your webdriver path (chromedriver.exe)

    chrome_options = Options()

    # Turn off Chrome notification and set language to English
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_experimental_option('prefs', {'intl.accept_languages': 'en,en_US'})

    # Start the service
    service = Service(webdriver_path)

    # Start the driver
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    return driver


def set_up_driver(url, service):
    '''Set up the driver and get url'''
    driver = configure_driver()

    # get url
    driver.get(url)
    return driver



In [13]:
def scroll_and_click_button(driver):
    '''Scroll down and click the button to load button "All comments| Most relevant,...'''
    cnt = 3
    try:
        # Scroll down until the button is found and clickable
        while True:
            if cnt <= 0:
                break
            try:
                # Locate the button using its class names
                button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "div.x9f619.x1n2onr6.x1ja2u2z.x6s0dn4.x3nfvp2.xxymvpz"))
                )
                button.click()
                print("Button clicked successfully.")
                break
            except Exception as e:
                # Scroll down a bit and try again
                driver.execute_script("window.scrollBy(0, 1000);")
                cnt -= 1
        return True
    except Exception as e:
        print("Unable to locate and click the button.", e)
        return False

def click_view_more_btn(driver):
    '''Click the view more button to change the show style of comments of a post'''

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Find the view more button
    view_more_btn = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'more comments')]")))
    view_more_btn.click()
    print("View more button clicked successfully.")
    return None

def click_showed_type_btn(driver, btn_name):
    '''Click the button to show the most relevant comments or all comments under a post by the argument btn_name'''

    try:
        # Scroll down to the button and click
        driver.execute_script("window.scrollTo(0, window.scrollY)")  # Scroll to the top

        # Find the button by its text
        most_relevant_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[contains(text(), '%s')]"%(btn_name))))
        most_relevant_button.click()
    except Exception as e:
        print("Can not click", btn_name)
        return False

def show_more_comments1(driver, time = 50):
    while True:
        try:
            click_view_more_btn(driver)
            time -= 1
            if time < 0:
                break
        except Exception as e:
            print("Out of contents")
            break

def show_more_comments(driver):
    '''Show more comments under a post'''

    # Limit the number of attempts to load more comments
    max_attempts = 10  # Adjust based on typical post length and your needs
    attempts = 0
    last_count = 0
    stable_attempts = 3  # Number of attempts with no new comments before considering end

    # Keep trying to load more comments until the limit is reached
    while attempts < max_attempts:
        try:
            click_view_more_btn(driver)
            time.sleep(2)  # Give some time for comments to load
            current_count = len(driver.find_elements(By.XPATH, "//div[contains(@class, 'x1n2onr6 x46jau6')]"))

            if current_count == last_count:
                stable_attempts -= 1
                if stable_attempts == 0:
                    print("No more comments to load.")
                    break
            else:
                last_count = current_count
                stable_attempts = 3  # Reset the stable_attempts counter

            attempts += 1
        except Exception as e:
            print("All comments loaded.")
            break

def show_all_replies(driver, threshold, ite):
    '''Show all replies of comments under a post
    Limited time: start - end = 3s => If there is no comment shown in 3s, stop
    Threshold: maximum number of comments'''
    arr = []
    cnt = 1
    while True:
        start = time.time() 
        if cnt > threshold: #Threshold
            break
        try:
            # Find replied comments
            all_replied_comments = driver.find_elements(By.XPATH, "//div[contains(@class, 'x1n2onr6 x46jau6')]")
        except:
            break
        
        # save count of comments to check if all comments are shown
        arr.append(len(all_replied_comments))
        if arr.count(max(arr)) >= ite:
            print("All replies are shown")
            return
        
        # Try to show sub-replied comments in replied-comments
        for comment in all_replied_comments:

            driver.execute_script("window.scrollBy(0, -50);")  # Scroll down 50px to load more comments
        
            try:
                view_more_buttons = WebDriverWait(comment, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'html-div xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x78zum5 x1iyjqo2 x21xpn4 x1n2onr6')]")))
                view_more_buttons.click()
                sub_cmt = comment.find_elements(By.XPATH, "//div[contains(@class, 'x1n2onr6 x1swvt13 x1iorvi4 x78zum5 x1q0g3np x1a2a7pz')]")
                for sub_cmt in comment:
                    try:
                        view_more_sub_buttons = WebDriverWait(sub_cmt, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'html-div xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x78zum5 x1iyjqo2 x21xpn4 x1n2onr6')]")))
                        view_more_sub_buttons.click()
                        #cnt += 1
                    except:
                        break

                cnt += 1

            except Exception as e:
                break

            finally:
                end = time.time()
                if end - start > 3:
                    #print("Finally")
                    return

def filter_spam(text):
    '''Filter spam comments based on user-defined keywords'''

    spam_text = ['http', 'miễn phí', '100%', 'kèo bóng', 'khóa học', 'netflix', 'Net Flix', 'shopee', 'lazada']
    for spam in spam_text:
        if spam in text.lower():
            return True
    return False

def get_comments(driver, limit_text=2500):
    '''Get comments under a post and filter spam comments
    Return:  - dataframe of comments: id, text, is_spam, tag_name.
             - number of comments'''
    cnt = 0
    treasured_comments = []
    is_spam = 0
    comments = driver.find_elements(By.XPATH, "//div[contains(@class, 'x1n2onr6 x1swvt13 x1iorvi4 x78zum5 x1q0g3np x1a2a7pz') or contains(@class, 'x1n2onr6 xurb0ha x1iorvi4 x78zum5 x1q0g3np x1a2a7pz')]")
    for comment in comments:
        try:
            # Check if comment contains text
            text_ele = comment.find_element(By.XPATH, ".//div[contains(@class, 'xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs')]")
            username = comment.find_element(By.XPATH, ".//span[@class='x3nfvp2']/span")

            if text_ele:
                try:
                    name_tag = text_ele.find_element(By.XPATH, ".//span[@class='xt0psk2']/span")
                    name_tag = name_tag.text
                except:
                    name_tag = None

                # Limit the number of comments  
                cnt += 1
                if cnt > limit_text:
                    break
                text = text_ele.text
                if cnt % 10 == 0:
                    print("Count: ", cnt)

                # Filter spam comments    
                if filter_spam(text):
                    is_spam = 1
                else:
                    is_spam = 0
                treasured_comments.append({
                    "id" : cnt,
                    "username": username.text,
                    "text": text,
                    'tag_name': name_tag,
                    'is_spam': is_spam
                })
        except Exception as e:
            continue
    print("Crawl successfully!!! \nTotal Comments: ", cnt)
    return treasured_comments, cnt

def get_url():
    '''Get the URL of a Facebook post from the user input'''
    #get url from input
    url = input("Enter the URL: ")
    return url


def save_to_csv(df, file_name):
    '''Save the dataframe to a CSV file'''
    df.to_csv(file_name, index=False)


In [21]:
def crawl(driver, url, threshold, ite):

    driver.get(url)
    if scroll_and_click_button(driver):
      click_showed_type_btn(driver, "All comments")

      #Show more comments
      try:
        show_more_comments1(driver, 100)
      except:
        print('No more cmt')

      # Scroll to top
      driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.CONTROL + Keys.HOME)

      # Show comment and scroll step by step
      driver.execute_script("window.scrollTo(0, 700);") #cuộn
      driver.execute_script("window.scrollTo(0, 700);")  #cuộn

      # Sleep 5s
      time.sleep(5)

      # Scroll để các button visible and show replies
      driver.execute_script("window.scrollTo(0, 900);")

      #Show replies
      try:
        show_all_replies(driver, threshold, ite)
      except Exception as e:
        print('Replies are limited', e)
        pass

      #driver.minimize_window()
      #driver.maximize_window()

      # Get comments
      cmts, cnt = get_comments(driver, 2500)

      return cmts, cnt
    else:
      print("No comment found!")
      return None


In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
'''
print("Attempting to start Chrome...")
try:
    options = webdriver.ChromeOptions()
    options.binary_location = r'E:/Users/1167486/Local/Drivers_web/chrome-win64/chrome.exe'
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--no-sandbox') # Often helpful in restricted environments
    options.add_argument('--disable-dev-shm-usage') # Overcomes resource limits
    options.add_argument('--headless') # Try headless to rule out display/GUI issues
    options.add_argument('--disable-gpu') # Often needed with headless

    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    print("Driver started. Navigating to Google...")
    driver.get("https://www.google.com")
    print(f"Page title: {driver.title}")
    driver.quit()
    print("Driver quit successfully.")
except Exception as e:
    print(f"An error occurred: {e}")
'''

'\nprint("Attempting to start Chrome...")\ntry:\n    options = webdriver.ChromeOptions()\n    options.binary_location = r\'E:/Users/1167486/Local/Drivers_web/chrome-win64/chrome.exe\'\n    options.add_argument(\'--disable-blink-features=AutomationControlled\')\n    options.add_argument(\'--no-sandbox\') # Often helpful in restricted environments\n    options.add_argument(\'--disable-dev-shm-usage\') # Overcomes resource limits\n    options.add_argument(\'--headless\') # Try headless to rule out display/GUI issues\n    options.add_argument(\'--disable-gpu\') # Often needed with headless\n\n    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)\n    print("Driver started. Navigating to Google...")\n    driver.get("https://www.google.com")\n    print(f"Page title: {driver.title}")\n    driver.quit()\n    print("Driver quit successfully.")\nexcept Exception as e:\n    print(f"An error occurred: {e}")\n'

In [16]:
options = webdriver.ChromeOptions()
options.binary_location = r'E:/Users/1167486/Local/Drivers_web/chrome-win64/chrome.exe'
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage') 
#options.add_argument('--headless') 
#options.add_argument('--disable-gpu') 
driverFB = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), 
                          options=options)


In [17]:
FACEBOOK_LOGIN_URL = 'https://www.facebook.com/login/'
driverFB.maximize_window()
driverFB.get(FACEBOOK_LOGIN_URL)
time.sleep(5)

In [27]:
fb_test ='https://www.facebook.com/revolutapp/posts/pfbid036Ex3ZyiYbApCH3mNzt6e8KxtnnoX2Cs9GCkn4SWc8sarJPYJTvnNTRaXVJ8A7MvDl'

In [28]:
cnt = 0
threshold = 60 
ite = 20 
url = fb_test
driver = driverFB

cmt_data, cnt = crawl(driver, url, threshold, ite)
cmt_data_json = json.dumps(cmt_data, ensure_ascii=False)



Button clicked successfully.
Can not click All comments
Out of contents
All replies are shown
Crawl successfully!!! 
Total Comments:  0


In [29]:
cmts = pd.read_json(StringIO(cmt_data_json), orient='records')


In [26]:
cmts.head(10)

""


In [ ]:
save_to_csv(cmts, 'E:/Users/1167486/Downloads/crawled_data.csv')
time.sleep(50)
driver.quit()